[View in Colaboratory](https://colab.research.google.com/github/ajoshiusc/deepReg/blob/master/deep_rigid_usc.ipynb)

In [0]:
#AUM
#Shree Ganeshaya Namaha
import tensorflow as tf
tf.test.gpu_device_name()
from skimage import transform as tf
from skimage.transform import resize, rotate
from skimage.color import rgb2gray
from skimage.io import imread
from keras.layers import Input,Conv2D,concatenate,MaxPooling2D,Flatten,Dense,Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras import backend as K
import numpy as np
from numpy.random import uniform
import matplotlib.pyplot as plt
from skimage.viewer import ImageViewer
import cv2
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter
import matplotlib.pyplot as plt

%matplotlib inline

from google.colab import files
uploaded = files.upload()


K.set_image_data_format('channels_last')  # TF dimension ordering in this code

from keras import losses
img_rows = 256
img_cols = 256


#taken from: https://www.kaggle.com/bguberfain/elastic-transform-for-data-augmentation
# Function to distort image
def elastic_transform(image, alpha, sigma, alpha_affine, random_state=None):
    """Elastic deformation of images as described in [Simard2003]_ (with modifications).
    .. [Simard2003] Simard, Steinkraus and Platt, "Best Practices for
         Convolutional Neural Networks applied to Visual Document Analysis", in
         Proc. of the International Conference on Document Analysis and
         Recognition, 2003.

     Based on https://gist.github.com/erniejunior/601cdf56d2b424757de5
    """
    if random_state is None:
        random_state = np.random.RandomState(None)

    shape = image.shape
    shape_size = shape[:2]
    
    # Random affine
    center_square = np.float32(shape_size) // 2
    square_size = min(shape_size) // 3
    pts1 = np.float32([center_square + square_size, [center_square[0]+square_size, center_square[1]-square_size], center_square - square_size])
    pts2 = pts1 + random_state.uniform(-alpha_affine, alpha_affine, size=pts1.shape).astype(np.float32)
#    M = cv2.getAffineTransform(pts1, pts2)
#    image = cv2.warpAffine(image, M, shape_size[::-1], borderMode=cv2.BORDER_REFLECT_101)

    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma) * alpha
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma) * alpha
    dz = np.zeros_like(dx)

    x, y = np.meshgrid(np.arange(shape[1]), np.arange(shape[0]))
    indices = np.reshape(y+dy, (-1, 1)), np.reshape(x+dx, (-1, 1))

    return map_coordinates(image, indices, order=1, mode='reflect').reshape(shape)



def get_rr_net():
    inputs = Input((img_rows, img_cols, 2))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4_1 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4_1)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
    flat1 = Flatten()(conv5)
    d1= Dense(512,activation='relu')(flat1)
    d2= Dense(64,activation='relu')(d1)

    out_theta = Dense(3)(d2)
#    conv_tx = Conv2D(1, (1, 1), activation=final_activation)(conv5)
#    conv_ty = Conv2D(1, (1, 1), activation=final_activation)(conv5)
#    conv_theta = Conv2D(1, (1, 1), activation='tanh')(conv5)

#    out_img = rotate(inputs,conv_theta)

    model = Model(inputs=[inputs], outputs=out_theta)

    model.compile(optimizer='adam', loss=losses.mean_squared_error, metrics=['mse'])

    return model


def gen_train_data(img, N=1024,  nodist=0):
    imgs_train = np.zeros((N, img.shape[0], img.shape[1], 2))
    noise = 0*uniform(low=-1,high=1,size=imgs_train.shape)
    out_train = np.zeros((N, 3))
    rot = uniform(low=-60, high=60, size=(N,1))
    tx = uniform(low=-100, high=100, size=(N,1))
    ty = uniform(low=-100, high=100, size=(N,1))
    out_train[:,0]=rot.squeeze()
    out_train[:,1]=tx.squeeze()
    out_train[:,2]=ty.squeeze()

    in_rot = uniform(low=-60, high=60, size=(N,1))
 
    for j in range(N):
      
#        img2 = tf.warp(img,aff)
        img2 = rotate(img, in_rot[j],mode='edge')
        img2 -= np.mean(img2)
        img2 /= np.std(img2)
        if nodist==0:
            img2 = elastic_transform(img2, img2.shape[1] * 1.2, img2.shape[1] * 0.08,img2.shape[1] * 0.08)
        
        imgs_train[j, :, :, 0] = img2 + (1.0-nodist)*noise[j,:,:,0]
       
        aff = tf.AffineTransform(rotation = (np.pi/180.0)*np.float(rot[j]), translation=(tx[j],ty[j]))

        #img3 = 10-1*np.tanh(img2) + 0*  noise[j,:,:,1]#
        img3 = 10-1*np.tanh(tf.warp(img2, aff, mode='edge')) +  (1.0-nodist)*noise[j,:,:,1]#
        
        if nodist == 0:
            img3 = elastic_transform(img3, img3.shape[1] * .2, img3.shape[1] * 0.08,img3.shape[1] * 0.08)

        img3 -= np.mean(img3)
        img3 /= np.std(img3)
        imgs_train[j, :, :, 1] = img3
        
        if 0:
            plt.imshow(img3)
            plt.show()
            
            plt.imshow(img2)
            plt.show()
             
    return imgs_train, out_train


def train_model():
    print('-'*30)
    print('Loading and preprocessing train data...')
    print('-'*30)
    img = resize(rgb2gray(imread('sample_brain.png')).astype('float32'),(img_rows,img_cols),mode='constant')
    mean = np.mean(img)  # mean for data centering
    std = np.std(img)  # std for data normalization

    img -= mean
    img /= std
    

    print('Creating and compiling model...')
    rrmodel = get_rr_net()
    rrmodel.load_weights('weights.h5')
    model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss', save_best_only=True)

    print('Fitting Model')
    for repind in range(100):
        imgs_train, out_train = gen_train_data(img, 512)
        history = rrmodel.fit(imgs_train, out_train, batch_size=64, epochs=5, verbose=1,
                              shuffle=True, validation_split=0.2,
                              callbacks=[model_checkpoint])

    # list all data in history
    print(history.history.keys())
    # summarize history for accuracy
    plt.plot(history.history['mean_squared_error'])
    plt.plot(history.history['val_mean_squared_error'])
    plt.title('model fit mse')
    plt.ylabel('mse')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    
    
def test_model():
    
    print('Test data...')
    img = resize(rgb2gray(imread('sample_brain.png')).astype('float32'),(img_rows,img_cols),mode='reflect')
    mean = np.mean(img)  # mean for data centering
    std = np.std(img)  # std for data normalization

    img -= mean
    img /= std

    rrmodel = get_rr_net()
    rrmodel.load_weights('weights.h5')

    imgs_test, out_test = gen_train_data(img, 4, nodist=1)
#    plt.imshow(np.absolute(imgs_test[0,:,:,0].squeeze()),cmap='gray')
#    plt.imshow(np.absolute(imgs_test[0,:,:,1].squeeze()),cmap='gray')

    pred_theta = rrmodel.predict(imgs_test, verbose=1)

    for jj in range(4):
        fig=plt.figure(figsize=(20,10)); 
        fig.add_subplot(1,3,1);ax = plt.gca(); ax.grid(False)
        plt.axis('off')
        plt.imshow(np.absolute(imgs_test[jj,:,:,0].squeeze()),cmap='gray')
        fig.add_subplot(1,3,2);ax = plt.gca(); ax.grid(False)
        plt.axis('off')
        plt.imshow(np.absolute(imgs_test[jj,:,:,1].squeeze()),cmap='gray')
        aff = tf.AffineTransform(rotation = (np.pi/180.0)*pred_theta[jj,0], translation=(pred_theta[jj,1],pred_theta[jj,2]))
        img2 = tf.warp(np.absolute(imgs_test[jj,:,:,0].squeeze()),aff)
        fig.add_subplot(1,3,3);ax = plt.gca(); ax.grid(False)
        plt.axis('off')
        plt.imshow(img2,cmap='gray')
        plt.show()
    
    
    print(pred_theta)
    print(out_test)



if __name__ == '__main__':
    train_model()
    test_model()


------------------------------
Loading and preprocessing train data...
------------------------------
Creating and compiling model...
Fitting Model
Train on 409 samples, validate on 103 samples
Epoch 1/5
409/409 [==============================] - 8s 19ms/step - loss: 102.3613 - mean_squared_error: 102.3613 - val_loss: 81.9868 - val_mean_squared_error: 81.9868
Epoch 2/5
409/409 [==============================] - 7s 16ms/step - loss: 65.1272 - mean_squared_error: 65.1272 - val_loss: 59.7503 - val_mean_squared_error: 59.7503
Epoch 3/5
409/409 [==============================] - 7s 16ms/step - loss: 43.9674 - mean_squared_error: 43.9674 - val_loss: 59.7712 - val_mean_squared_error: 59.7712
Epoch 4/5
409/409 [==============================] - 7s 16ms/step - loss: 28.3346 - mean_squared_error: 28.3346 - val_loss: 42.3593 - val_mean_squared_error: 42.3593
Epoch 5/5
409/409 [==============================] - 7s 16ms/step - loss: 21.7539 - mean_squared_error: 21.7539 - val_loss: 43.8995 - val_me

409/409 [==============================] - 7s 16ms/step - loss: 39.1994 - mean_squared_error: 39.1994 - val_loss: 39.6100 - val_mean_squared_error: 39.6100
Epoch 2/5
409/409 [==============================] - 7s 16ms/step - loss: 24.0135 - mean_squared_error: 24.0135 - val_loss: 32.9875 - val_mean_squared_error: 32.9875
Epoch 3/5
409/409 [==============================] - 7s 16ms/step - loss: 16.5942 - mean_squared_error: 16.5942 - val_loss: 37.9980 - val_mean_squared_error: 37.9980
Epoch 4/5
409/409 [==============================] - 7s 16ms/step - loss: 11.5257 - mean_squared_error: 11.5257 - val_loss: 36.9954 - val_mean_squared_error: 36.9954
Epoch 5/5
409/409 [==============================] - 7s 16ms/step - loss: 9.4178 - mean_squared_error: 9.4178 - val_loss: 33.1887 - val_mean_squared_error: 33.1887
Train on 409 samples, validate on 103 samples
Epoch 1/5
409/409 [==============================] - 7s 16ms/step - loss: 36.7060 - mean_squared_error: 36.7060 - val_loss: 55.1298 - va

409/409 [==============================] - 7s 16ms/step - loss: 27.1918 - mean_squared_error: 27.1918 - val_loss: 42.2165 - val_mean_squared_error: 42.2165
Epoch 3/5
409/409 [==============================] - 7s 16ms/step - loss: 22.0735 - mean_squared_error: 22.0735 - val_loss: 44.2360 - val_mean_squared_error: 44.2360
Epoch 4/5
409/409 [==============================] - 7s 16ms/step - loss: 15.9334 - mean_squared_error: 15.9334 - val_loss: 44.8431 - val_mean_squared_error: 44.8431
Epoch 5/5
409/409 [==============================] - 7s 16ms/step - loss: 13.2898 - mean_squared_error: 13.2898 - val_loss: 38.6544 - val_mean_squared_error: 38.6544
Train on 409 samples, validate on 103 samples
Epoch 1/5
409/409 [==============================] - 7s 16ms/step - loss: 82.4112 - mean_squared_error: 82.4112 - val_loss: 92.9729 - val_mean_squared_error: 92.9729
Epoch 2/5
409/409 [==============================] - 7s 16ms/step - loss: 63.5481 - mean_squared_error: 63.5481 - val_loss: 56.3151 - 

409/409 [==============================] - 7s 16ms/step - loss: 39.5246 - mean_squared_error: 39.5246 - val_loss: 58.9982 - val_mean_squared_error: 58.9982
Epoch 4/5
409/409 [==============================] - 7s 16ms/step - loss: 35.1501 - mean_squared_error: 35.1501 - val_loss: 69.1926 - val_mean_squared_error: 69.1926
Epoch 5/5
409/409 [==============================] - 7s 16ms/step - loss: 23.8931 - mean_squared_error: 23.8931 - val_loss: 56.5952 - val_mean_squared_error: 56.5952
Train on 409 samples, validate on 103 samples
Epoch 1/5
409/409 [==============================] - 7s 16ms/step - loss: 69.2175 - mean_squared_error: 69.2175 - val_loss: 54.6757 - val_mean_squared_error: 54.6757
Epoch 2/5
409/409 [==============================] - 7s 16ms/step - loss: 43.4028 - mean_squared_error: 43.4028 - val_loss: 52.4730 - val_mean_squared_error: 52.4730
Epoch 3/5
409/409 [==============================] - 7s 16ms/step - loss: 24.9461 - mean_squared_error: 24.9461 - val_loss: 49.6789 - 

409/409 [==============================] - 7s 16ms/step - loss: 23.8503 - mean_squared_error: 23.8503 - val_loss: 53.0529 - val_mean_squared_error: 53.0529
Epoch 5/5
409/409 [==============================] - 7s 16ms/step - loss: 15.6523 - mean_squared_error: 15.6523 - val_loss: 53.5502 - val_mean_squared_error: 53.5502
Train on 409 samples, validate on 103 samples
Epoch 1/5
409/409 [==============================] - 7s 16ms/step - loss: 57.8403 - mean_squared_error: 57.8403 - val_loss: 83.9337 - val_mean_squared_error: 83.9337
Epoch 2/5
409/409 [==============================] - 7s 16ms/step - loss: 44.0493 - mean_squared_error: 44.0493 - val_loss: 65.4902 - val_mean_squared_error: 65.4902
Epoch 3/5
409/409 [==============================] - 7s 16ms/step - loss: 29.4706 - mean_squared_error: 29.4706 - val_loss: 58.0521 - val_mean_squared_error: 58.0521
Epoch 4/5
409/409 [==============================] - 7s 16ms/step - loss: 19.8618 - mean_squared_error: 19.8618 - val_loss: 52.2183 - 

...

409/409 [==============================] - 7s 16ms/step - loss: 35.3837 - mean_squared_error: 35.3837 - val_loss: 34.4607 - val_mean_squared_error: 34.4607
Epoch 2/5
409/409 [==============================] - 7s 16ms/step - loss: 24.3687 - mean_squared_error: 24.3687 - val_loss: 40.9372 - val_mean_squared_error: 40.9372
Epoch 3/5
409/409 [==============================] - 7s 16ms/step - loss: 17.8659 - mean_squared_error: 17.8659 - val_loss: 37.7152 - val_mean_squared_error: 37.7152
Epoch 4/5
409/409 [==============================] - 7s 16ms/step - loss: 20.1588 - mean_squared_error: 20.1588 - val_loss: 44.8217 - val_mean_squared_error: 44.8217
Epoch 5/5
409/409 [==============================] - 7s 16ms/step - loss: 14.4120 - mean_squared_error: 14.4120 - val_loss: 37.7476 - val_mean_squared_error: 37.7476
Train on 409 samples, validate on 103 samples
Epoch 1/5
409/409 [==============================] - 7s 16ms/step - loss: 37.0668 - mean_squared_error: 37.0668 - val_loss: 44.9099 - 

Epoch 3/5
409/409 [==============================] - 7s 16ms/step - loss: 19.5514 - mean_squared_error: 19.5514 - val_loss: 39.8050 - val_mean_squared_error: 39.8050
Epoch 4/5
409/409 [==============================] - 7s 16ms/step - loss: 16.4775 - mean_squared_error: 16.4775 - val_loss: 40.5216 - val_mean_squared_error: 40.5216
Epoch 5/5
409/409 [==============================] - 7s 16ms/step - loss: 16.5181 - mean_squared_error: 16.5181 - val_loss: 42.8840 - val_mean_squared_error: 42.8840
Train on 409 samples, validate on 103 samples
Epoch 1/5
409/409 [==============================] - 7s 16ms/step - loss: 40.0314 - mean_squared_error: 40.0314 - val_loss: 42.5068 - val_mean_squared_error: 42.5068
Epoch 2/5
409/409 [==============================] - 7s 16ms/step - loss: 23.7167 - mean_squared_error: 23.7167 - val_loss: 47.4087 - val_mean_squared_error: 47.4087
Epoch 3/5
409/409 [==============================] - 7s 16ms/step - loss: 16.2439 - mean_squared_error: 16.2439 - val_loss: 

409/409 [==============================] - 7s 16ms/step - loss: 18.5579 - mean_squared_error: 18.5579 - val_loss: 44.5964 - val_mean_squared_error: 44.5964
Epoch 5/5
409/409 [==============================] - 7s 16ms/step - loss: 13.3697 - mean_squared_error: 13.3697 - val_loss: 40.9240 - val_mean_squared_error: 40.9240
Train on 409 samples, validate on 103 samples
Epoch 1/5
409/409 [==============================] - 7s 16ms/step - loss: 45.2339 - mean_squared_error: 45.2339 - val_loss: 70.8793 - val_mean_squared_error: 70.8793
Epoch 2/5
409/409 [==============================] - 7s 16ms/step - loss: 28.9942 - mean_squared_error: 28.9942 - val_loss: 67.3223 - val_mean_squared_error: 67.3223
Epoch 3/5
409/409 [==============================] - 7s 16ms/step - loss: 19.5145 - mean_squared_error: 19.5145 - val_loss: 60.8728 - val_mean_squared_error: 60.8728
Epoch 4/5
409/409 [==============================] - 7s 16ms/step - loss: 17.2909 - mean_squared_error: 17.2909 - val_loss: 48.3652 - 

409/409 [==============================] - 7s 16ms/step - loss: 6.3667 - mean_squared_error: 6.3667 - val_loss: 36.3506 - val_mean_squared_error: 36.3506
Train on 409 samples, validate on 103 samples
Epoch 1/5
409/409 [==============================] - 7s 16ms/step - loss: 43.9419 - mean_squared_error: 43.9419 - val_loss: 39.8702 - val_mean_squared_error: 39.8702
Epoch 2/5
409/409 [==============================] - 7s 16ms/step - loss: 28.9270 - mean_squared_error: 28.9270 - val_loss: 41.6068 - val_mean_squared_error: 41.6068
Epoch 3/5
409/409 [==============================] - 7s 16ms/step - loss: 18.4455 - mean_squared_error: 18.4455 - val_loss: 43.1888 - val_mean_squared_error: 43.1888
Epoch 4/5
409/409 [==============================] - 7s 16ms/step - loss: 14.2503 - mean_squared_error: 14.2503 - val_loss: 36.7818 - val_mean_squared_error: 36.7818
Epoch 5/5
409/409 [==============================] - 7s 16ms/step - loss: 10.4952 - mean_squared_error: 10.4952 - val_loss: 37.1428 - va

409/409 [==============================] - 7s 16ms/step - loss: 36.5642 - mean_squared_error: 36.5642 - val_loss: 32.3167 - val_mean_squared_error: 32.3167


In [5]:
from google.colab import files
files.download('weights.h5')

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 48978, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 696, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil